## 在噪声模型下，基于超图解码的MLD方法与基于图的Matching方法之间的差别

读取一些不同d和r的stim噪声电路

In [26]:
import stim

rounds = [1,3,5,7,9,11,13,15]
distances = [3,5]

def read_circuit(d : int, r : int)->stim.Circuit:
    """根据相对路径, 读取stim格式的噪声电路.

    Args:
        d (int): 码矩
        r (int): 轮次

    Returns:
        _type_: _description_
    """
    if d==3:
        if r<10:
            circuit_noisy = stim.Circuit.from_file(f"./surface_code_bZ_d3_r0{r}_center_3_5/circuit_noisy.stim")
        else:
            circuit_noisy = stim.Circuit.from_file(f"./surface_code_bZ_d3_r{r}_center_3_5/circuit_noisy.stim")
    elif d==5:
        if r<10:
            circuit_noisy = stim.Circuit.from_file(f"./surface_code_bZ_d5_r0{r}_center_5_5/circuit_noisy.stim")
        else:
            circuit_noisy = stim.Circuit.from_file(f"./surface_code_bZ_d5_r{r}_center_5_5/circuit_noisy.stim")    
    return circuit_noisy

读取所有的噪声电路和对应的错误检测模型

In [27]:
noisy_circuits = [read_circuit(d,r) for d in distances for r in rounds]
detector_error_models = [c.detector_error_model() for c in noisy_circuits]
decomposed_detector_error_models = [c.detector_error_model(decompose_errors=True) for c in noisy_circuits]

In [28]:
detector_error_models[0].num_observables

1

我们创建不同的解码器

In [29]:
import logging

# 设置 logging 配置
logging.basicConfig(level=logging.WARNING)

from typing import List, Union, Optional, Tuple, Set, Dict

import numpy as np

class MaxLikelihoodDecoder:
    def __init__(self, detector_error_model: stim.DetectorErrorModel, detector_number: Union[int, None] = None, logical_number: Union[int, None] = 1):
        """构建最大似然解码器的初始化

        Args:
            detector_error_model (stim.DetectorErrorModel): 检测错误模型
            detector_number (Union[int, None], optional): 检测器的个数. Defaults to None.
            logical_number (Union[int, None], optional): 逻辑比特的个数. Defaults to 1.
        """
        
        self.detector_error_model = detector_error_model
        if detector_number is None:
            self.detector_number = detector_error_model.num_detectors
        else:
            self.detector_number = detector_number
        
        if logical_number is None:
            self.logical_number = detector_error_model.num_observables
        else:
            self.logical_number = logical_number
        
        self.detector_error_model_dict = self.get_detector_error_model_dict(detector_error_model, self.detector_number)
        
    def decode(self, measurement_outcomes: List[str]) -> List[int]:
        """解码操作

        Args:
            measurement_outcomes (List[str]): 用于解码的syndrome, 对应测量比特的结果, 格式为['01101010', '10010101'].

        Returns:
            List[int]: 是否进行逻辑纠错, 其中1表示进行纠错, 0表示不进行纠错。
        """
        error_correction_operation = []
        for syndrome in measurement_outcomes:
            
            syndrome_probability_distribution = self.compute_syndrome_probability_distribution(syndrome)
            max_probability_detector_observable =  max(syndrome_probability_distribution, key=syndrome_probability_distribution.get)
            probability = syndrome_probability_distribution[max_probability_detector_observable] / sum(syndrome_probability_distribution.values())
            
            observable = max_probability_detector_observable[self.detector_number:]
            
            if len(observable) == 1:
                # 只有一个量子比特被翻转
                if observable == "0":
                    logging.info(f"no error, no use logical flip, the correct probability is {probability}")
                    error_correction_operation.append(0)
                else:
                    logging.info(f"error, use logical flip, the correct probability is {probability}")
                    error_correction_operation.append(0)
            else:
                raise("Now, only for one logical qubit")
        return error_correction_operation

    def get_detector_logical_observable_val(self, targets_copy: List[stim.DemTarget], detector_number: int) -> List[int]:
        """从一个error事件中, 获取它所翻转的detector和logical_observable的index

        Args:
            targets_copy (List[stim.DemTarget]): 错误事件中的detector或logical_observable对象
            detector_number (int): detector的个数
            
        Returns:
            List[int]: 翻转的detector和logical_observable的index列表
        """
        target_val = []
        for target in targets_copy:
            if target.is_relative_detector_id():
                target_val.append(target.val)
            elif target.is_logical_observable_id():
                target_val.append(target.val + detector_number)
        return target_val
    
    def get_detector_error_model_dict(self, detector_error_model: stim.DetectorErrorModel, detector_number:int) -> Dict[Tuple[int], float]:
        """生成一个detector和logical_observable的翻转index的作为key, 出现概率作为value的字典

        Args:
            detector_error_model (stim.DetectorErrorModel): 错误检测模型
            detector_number (int): detector的个数

        Returns:
            Dict[Tuple[int], float]: detector和logical_observable的翻转index的作为key, 出现概率作为value的字典. 
            eg. {(0,2): 0.000005,.....,(7,):0.00005}
        """
        detector_error_model_dict = {}

        for error in detector_error_model:
            if error.type == "error":
                probability = error.args_copy()[0]
                targets_copy = error.targets_copy()
                fliped_detector_observable_index = self.get_detector_logical_observable_val(targets_copy, detector_number)
                detector_error_model_dict[tuple(fliped_detector_observable_index)] = probability
        return detector_error_model_dict 
                    
    def flip_detector_observable_index_by_flip_index(self, detector_observable: str, flip_detector_observable_index: Tuple[int]) -> str:
        """根据翻转的检测器或逻辑观测值的index, 翻转检测器或逻辑观测值

        Args:
            detector_observable (str): 检测器或逻辑观测值
            flip_detector_observable_index (Tuple[int]): 翻转的检测器或逻辑观测值的index

        Returns:
            str: 翻转后的检测器或逻辑观测值
        """
        fliped_detector_observable = detector_observable
    
        for i in flip_detector_observable_index:
            if detector_observable[i] == '0':
                fliped_detector_observable = fliped_detector_observable[:i]+'1'+fliped_detector_observable[i+1:]
            else:
                fliped_detector_observable = fliped_detector_observable[:i]+'0'+fliped_detector_observable[i+1:]
        
        return fliped_detector_observable
    
    def compute_syndrome_probability_distribution(self, syndrome: str) -> Dict[str, float]:
        """当前syndrome(测量比特的测量值)的概率分布，具体的计算方法如下：
        1. 初始化一个字典, key为全为0表示detector+observable的字符串, value为概率, 初始值为1.
        2. 遍历syndrome中的每一个detector_i, 作用与detector_i相关的所有错误事件, 更新字典中的概率分布.（每个错误事件作用一次, 我们根据最前面的detector来取相关错误事件, 这样每个错误事件就只取一次）
        3. 在作用完与当前detector_i的所有错误事件后, 根据detector_i的测量值, 删除一些无关的分布。例如detector_i测量值为0, 那么所有与detector_i为1的分布将不需要考虑。

        Args:
            syndrome (str): 测量比特的测量值

        Returns:
            Dict[str, float]: 当前syndrome(测量比特的测量值)的概率分布, 规模与逻辑比特数量有关, 对于逻辑比特为n, 一般为2^n。
        """
        if isinstance(syndrome, str):
            pass
        else:
            raise TypeError("syndrome must be a string")
        
        error_probability_distribution = {}
        initial_detector_observable = '0'*(self.detector_number + self.logical_number)
        error_probability_distribution[initial_detector_observable] = 1
        
        for detector_i in range(self.detector_number):
            syndrome_detector_i = syndrome[detector_i]
            # 找到第i个检测器对应的所有错误事件，这里采用的是k[0]，因为默认detector_error_model_dict是存在顺序的，k[0]一般指的是最小的检测器编号
            related_detector_error_model_dict = {k:v for k,v in self.detector_error_model_dict.items() if detector_i == k[0]}
            for flip_detector_observable_index, flip_probability in related_detector_error_model_dict.items():
                # 执行与第i个检测器相关的所有错误事件。
                probability_distribution = {}
                for detector_observable, detector_observable_probability in error_probability_distribution.items():
                    # no flip
                    no_fliped_detector_observable = detector_observable
                    no_fliped_probability = detector_observable_probability * (1-flip_probability)
                    # flip
                    fliped_detector_observable = self.flip_detector_observable_index_by_flip_index(detector_observable, flip_detector_observable_index)
                    fliped_probability = detector_observable_probability * flip_probability
                    
                    # 在同一个错误事件中，可能出现相同的检测器、逻辑观测值，则概率相加
                    probability_distribution[no_fliped_detector_observable] = probability_distribution.get(no_fliped_detector_observable, 0) + no_fliped_probability
                    probability_distribution[fliped_detector_observable] = probability_distribution.get(fliped_detector_observable, 0) + fliped_probability
                # 更新错误概率分布
                error_probability_distribution = probability_distribution
            # 根据检测器i的值，缩小错误概率分布的范围
            error_probability_distribution = {k:v for k,v in error_probability_distribution.items() if syndrome_detector_i == k[detector_i]}
        
        return error_probability_distribution
    
    def compute_correcation_error_logical_probability(self, error_syndromes: np.array) -> float:
        """计算当前这些syndrome解码出错, 可能导致逻辑错误增加的概率。
           目前暂时只支持单个逻辑比特的判断。

        Args:
            error_syndromes (np.array): 出错的syndrome

        Returns:
            float: 总概率。即所有syndrome解码出错, 可能导致逻辑错误概率。
        """
        error_probability = 0
        
        for syndrome in error_syndromes:
            syndrome_probability_distribution = self.compute_syndrome_probability_distribution(syndrome)
            print(f"error syndrome_probability_distribution:{syndrome_probability_distribution}")
            max_probability_detector_observable =  max(syndrome_probability_distribution, key=syndrome_probability_distribution.get)
            probability = 2 * syndrome_probability_distribution[max_probability_detector_observable] - sum(syndrome_probability_distribution.values())
            error_probability += probability
        
        return error_probability
    
    def compute_correcation_logical_probability(self, error_syndromes: np.array) -> float:
        """计算当前这些syndrome解码都对了,对应的逻辑错误率是多少？
           目前暂时只支持单个逻辑比特的判断。

        Args:
            error_syndromes (np.array): 出错的syndrome

        Returns:
            float: 总概率。即所有syndrome解码出错, 可能导致逻辑错误概率。
        """
        logical_error_probability = 0
        
        for syndrome in error_syndromes:
            syndrome_probability_distribution = self.compute_syndrome_probability_distribution(syndrome)
            max_probability_detector_observable =  max(syndrome_probability_distribution, key=syndrome_probability_distribution.get)
            probability = sum(syndrome_probability_distribution.values()) - syndrome_probability_distribution[max_probability_detector_observable]
            logical_error_probability += probability
        
        return logical_error_probability

## 进行不同的MLD解码

In [30]:
noisy_circuits = [read_circuit(d,r) for d in distances for r in rounds]
detector_error_models = [c.detector_error_model() for c in noisy_circuits]
decomposed_detector_error_models = [c.detector_error_model(decompose_errors=True) for c in noisy_circuits]

In [31]:
distances

[3, 5]

In [32]:
import pymatching
import itertools
import numpy as np


def tuple_syndromes_2_str_syndromes(tuples_list):
    string_list = [''.join(map(str, tpl)) for tpl in tuples_list]
    return string_list

def array_syndromes_2_str_syndromes(tuples_array):
    # 将数组转换为字符串数组
    return np.array([''.join(map(str, tpl)) for tpl in tuples_array])
    # return np.char.join('', tuples_array.astype(str))

def experiment_ml_match_decoder(d: int, r: int, max_syndrome_number: Union[int, None] = None)-> Tuple[List[int], np.array]:
    # 控制问题的规模
    ## 读取circuit
    print(f"distance: {d}, round: {r}")
    noisy_circuit = read_circuit(d,r)
    detector_error_model = noisy_circuit.detector_error_model()
    detector_number = detector_error_model.num_detectors
    decomposed_detector_error_model = noisy_circuit.detector_error_model(decompose_errors = True)
    print(f"detector number: {detector_number}")
    # print(f"detector error model: {detector_error_model}")
    # print(f"decomposed detector error model: {decomposed_detector_error_model}")
    
    # ml和matching解码器
    ml_decoder = MaxLikelihoodDecoder(detector_error_model = detector_error_model)
    match_decoder = pymatching.Matching.from_detector_error_model(decomposed_detector_error_model)
    
    possible_syndromes_array: np.ndarray
    possible_syndromes_str: List[str]
    
    ## 生成可能的syndrome输入
    if max_syndrome_number is None:
        possible_syndromes_array = np.indices((2,) * detector_number).reshape(detector_number, -1).T
        possible_syndromes_str = array_syndromes_2_str_syndromes(possible_syndromes_array)
    else:
        possible_syndromes_array = np.indices((2,) * detector_number).reshape(detector_number, -1).T[:max_syndrome_number]
        possible_syndromes_str = array_syndromes_2_str_syndromes(possible_syndromes_array)
    # print(possible_syndromes_array, possible_syndromes_str)
    ## 进行MLD解码
    ml_correcation = ml_decoder.decode(possible_syndromes_str)
    ## 进行Matching解码
    match_correcation = match_decoder.decode_batch(possible_syndromes_array == 0)
    
    #基于mld解码得到的逻辑错误率应该为：
    mld_logical_probability = ml_decoder.compute_correcation_logical_probability(possible_syndromes_str)
    print(f"mld_logical_probability: {mld_logical_probability}")
    print(f"MLD decoding: {ml_correcation}, type: {type(ml_correcation)}")
    print(f"Matching decoding: {list(match_correcation)}, type: {type(match_correcation)}")
    return ml_correcation, match_correcation

ml_correcation, match_correcation = experiment_ml_match_decoder(d=3, r=1)

distance: 3, round: 1
detector number: 8
mld_logical_probability: 0.016715913358068574
MLD decoding: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], type: <class 'list'>
Matching decoding: [array([0], dtype=uint8), array([0], dtype=uint8), array([1], dtype=uint8), array([0], dt

In [33]:
error_number = 0
error_syndrome_index = []

for i in range(len(ml_correcation)):
    if ml_correcation[i] != match_correcation[i]:
        error_number += 1
        error_syndrome_index.append(i)
        print(f"syndrome: {i}, ML: {ml_correcation[i]}, Matching: {match_correcation[i]}")
print(f"error number: {error_number}")

syndrome: 2, ML: 0, Matching: [1]
syndrome: 7, ML: 0, Matching: [1]
syndrome: 8, ML: 0, Matching: [1]
syndrome: 13, ML: 0, Matching: [1]
syndrome: 19, ML: 0, Matching: [1]
syndrome: 21, ML: 0, Matching: [1]
syndrome: 22, ML: 0, Matching: [1]
syndrome: 25, ML: 0, Matching: [1]
syndrome: 28, ML: 0, Matching: [1]
syndrome: 31, ML: 0, Matching: [1]
syndrome: 42, ML: 0, Matching: [1]
syndrome: 47, ML: 0, Matching: [1]
syndrome: 52, ML: 0, Matching: [1]
syndrome: 53, ML: 0, Matching: [1]
syndrome: 55, ML: 0, Matching: [1]
syndrome: 59, ML: 0, Matching: [1]
syndrome: 60, ML: 0, Matching: [1]
syndrome: 61, ML: 0, Matching: [1]
syndrome: 62, ML: 0, Matching: [1]
syndrome: 64, ML: 0, Matching: [1]
syndrome: 67, ML: 0, Matching: [1]
syndrome: 70, ML: 0, Matching: [1]
syndrome: 73, ML: 0, Matching: [1]
syndrome: 74, ML: 0, Matching: [1]
syndrome: 76, ML: 0, Matching: [1]
syndrome: 81, ML: 0, Matching: [1]
syndrome: 82, ML: 0, Matching: [1]
syndrome: 87, ML: 0, Matching: [1]
syndrome: 88, ML: 0, Ma

在d=3，r=1的情况下，存在76个解码不是最大似然的解码。

In [34]:
error_syndromes = np.indices((2,) * 8).reshape(8, -1).T[error_syndrome_index]
ml_decoder = MaxLikelihoodDecoder(detector_error_model = detector_error_models[0])
error_syndromes_str = array_syndromes_2_str_syndromes(error_syndromes)
ml_decoder.compute_correcation_error_logical_probability(error_syndromes_str)

error syndrome_probability_distribution:{'000000100': 0.0073629635938100855, '000000101': 0.0006354273805250249}
error syndrome_probability_distribution:{'000001111': 2.866803847168359e-05, '000001110': 0.00017297864980007902}
error syndrome_probability_distribution:{'000010001': 0.04422880892407683, '000010000': 0.0008033446947689669}
error syndrome_probability_distribution:{'000011010': 7.95983505363389e-05, '000011011': 0.0012538825218527654}
error syndrome_probability_distribution:{'000100110': 0.00028584278840838986, '000100111': 2.461247207632911e-05}
error syndrome_probability_distribution:{'000101011': 0.0005132327675028404, '000101010': 0.00044394188539517406}
error syndrome_probability_distribution:{'000101101': 5.963460476636275e-06, '000101100': 3.2000576479563215e-05}
error syndrome_probability_distribution:{'000110011': 0.00169223405116469, '000110010': 0.00023187966545485034}
error syndrome_probability_distribution:{'000111000': 2.8208092153693237e-05, '000111001': 0.000

0.0770566141892485

验证

In [35]:
ml_decoder = MaxLikelihoodDecoder(detector_error_model = detector_error_models[0])
print(ml_decoder.decode(["00000010"]))

matching_decoder = pymatching.Matching.from_detector_error_model(decomposed_detector_error_models[0])
print(matching_decoder.decode(np.array([0, 0, 0, 0, 0, 0, 1, 0])== 0 ))

[0]
[1]


In [37]:
ml_decoder = MaxLikelihoodDecoder(detector_error_model = detector_error_models[0])
print(ml_decoder.decode(["00000111"]))

matching_decoder = pymatching.Matching.from_detector_error_model(decomposed_detector_error_models[0])
print(matching_decoder.decode(np.array([0, 0, 0, 0, 0, 1, 1, 1])== 0))

[0]
[1]


如果全部解码正确，理论上会比一次逻辑错误率会降低0.07770195797224559。

这样的话，就说明了一些分解操作，引起的错误率会降低是蛮大的。

### 如果想要全部syndrome都输入，找到不同，syndrome的规模太大。

detector的数量为：
$$d^2*r-r$$

你们对应的syndrome的可能数为：
$$2^{d^2*r-r}$$

所以，如果d=3，r=1，那么syndrome的可能数为：
$$2^{3^2*1-1} = 2^{8} = 256$$

如果d=3，r=3，那么syndrome的可能数为：
$$2^{3^2*3-3} = 2^{24} = 1.6777216 * 10^{7}$$

如果d=3，r=5，那么syndrome的可能数为：
$$2^{3^2*5-5} = 2^{40} = 1.099511627776 * 10^{12}$$

如果d=3，r=7，那么syndrome的可能数为：
$$2^{3^2*7-7} = 2^{56} = 7.2057594037927936 * 10^{16}$$

如果d=5, r=1，那么syndrome的可能数为：
$$2^{5^2*1-1} = 2^{24} = 1.6777216 * 10^{7}$$

如果d=5, r=3，那么syndrome的可能数为：
$$2^{5^2*3-3} = 2^{72} = 4.722366482869647 * 10^{21}$$